## Raspagem dos resultados das provas da Maratona do Rio 2019 
Meia Maratona, Maratona e Desafio Cidade Maravilhosa

In [1]:
import os
import mechanize
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from time import time
from time import sleep
from random import randint
import json

In [2]:
url_base = 'https://globalcronometragem.com.br/evento/'
DATASET_FOLDER  = 'dataset'

eventos = ["meiadorio2019", "maratonadorio2019", "desafiocidademaravilhosa2019"]

br = mechanize.Browser()
br.addheaders = [('User-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.63 Safari/535.7')]

### Funções Utilitárias

In [3]:
# Recupera a listagem de categorias da prova do evento
def get_categorias(evento):
    
    #Controle de quantidade de requisição
    inicio = time()
    sleep(randint(1,5))
    espera = time() - inicio

    print('--> Request Categoria ' + evento + ' ' + str(espera))
    
    br.open(url_base + evento)
    html = br.response().read()
    
    cat_soup = BeautifulSoup(html, "html.parser")
    select_categorias = cat_soup.find(class_ = "form-control campocategoria")
    
    if select_categorias is None:
        return None
    
    options_categorias = select_categorias.find_all("option")
    
    if options_categorias is None:
        return None
    
    options_categorias = options_categorias[1:]
    
    categorias = []
    
    for o in options_categorias:
        categorias.append(o['value'])
    
    return categorias

In [4]:
#Recupera a URL para o resultado individual de cada atleta do evento por categoria
def get_url_resultados_individuais(lista_categorias, evento, debug = False):
    
    url_resultados_individuais = []

    for categoria in lista_categorias:

        #Controle de quantidade de requisição
        inicio = time()
        sleep(randint(1,5))
        espera = time() - inicio

        print('--> Request ' + categoria + ' ' + str(espera))
        
        params = {'npeito': '', 'campopesquisa': '', 'categoria': categoria}

        data = urllib.parse.urlencode(params)

        br.open(url_base + evento, data = data)
        resp = br.response().read()
        soup = BeautifulSoup(resp, "html.parser")

        div_resultados = soup.find_all(class_ = "campos")

        if div_resultados is None:
            continue

        for resultado in div_resultados:
            a = resultado.find("a")
            
            if a is None:
                continue;
                
            url_individual = a['href']
            
            url_resultados_individuais.append({
                "url": url_individual,
                "categoria": categoria
            })
    
        if debug:
            break
    
    return url_resultados_individuais

In [5]:
#Recupera o resultado individual do atleta
def get_resultado_individual_detalhado(url):

    br.open(url)
    html = br.response().read()

    soup = BeautifulSoup(html, "html.parser")

    div_externa = soup.find_all(class_ = "cimaevento span12")
    
    if div_externa is None:
        return
    
    if len(div_externa) < 2:
        return
    
    div = div_externa[1]

    temp = div.text.split('\n')

    temp = list(filter(lambda x: x != '', temp))[1:-1]

    temp_dic = dict(zip(temp[::2], temp[1::2]))
    
    return temp_dic

In [6]:
#Faz a raspagem de dados do evento e retorna em um dataframe Pandas
def scrape_evento(evento, debug = False):
    
    categorias = get_categorias(evento)

    resultados_categoria = get_url_resultados_individuais(categorias, evento, debug=debug)

    data = []
    count = 0
    
    for linha in resultados_categoria:
        
        if debug == True and count > 5:
            break
        
        #Controle de quantidade de requisição
        inicio = time()
        sleep(randint(1,5))
        espera = time() - inicio

        print('Request ' + str(count) + ' scrape ' + evento + ' ' + str(espera))
        
        count += 1
        
        resultado_individual = get_resultado_individual_detalhado(linha['url'])
        resultado_individual['Categoria'] = linha['categoria']
        resultado_individual['Colocação Geral'] = resultado_individual['Colocação Geral'].split(' ')[0]
        
        data.append(list(resultado_individual.values()))


    columns = ['Nome', 'N. Peito', 'Equipe', 'Modalidade', 'Sexo', 'Colocação Geral', 'Colocação faixa etária','Tempo bruto', 'Tempo líquido', 'Categoria']

    df = pd.DataFrame(data=data, columns=columns)
    
    return df

## Teste salvando resultados temporários

In [7]:
def get_baixados(evento):

    baixados = []

    for r, d, f in os.walk(DATASET_FOLDER + "/" + evento + "/resultado/"):
        baixados = [x.replace('.csv', '') for x in f]

    return baixados

In [8]:
def salva_urls_resultados(debug):
    for evento in eventos:
        categorias = get_categorias(evento)
        resultados_categoria = get_url_resultados_individuais(categorias, evento, debug)

        f = open(DATASET_FOLDER + "/" + evento + "/urls-resultados.txt", 'w')
        f.write(json.dumps(resultados_categoria))
        f.close()

In [ ]:
def salva_resultado_individual(debug = True, qtd = 5):

    for evento in eventos:

        print(evento)
        
        f = open(DATASET_FOLDER + "/" + evento + "/urls-resultados.txt", 'r')

        resultados_categoria = json.loads(f.read())

        f.close()

        count = 0

        print("Procurando resultado novo para baixar...")
        
        for linha in resultados_categoria:

            if count >= qtd:
                break
                
            baixados = get_baixados(evento)

            nome_arq_ind = linha['url'][linha['url'].find('?id=') + 4:] 

            if nome_arq_ind in baixados:
                #print(" " + nome_arq_ind + " já baixado.")
                continue
            
            print(" baixando" + nome_arq_ind)

            data = []

            #Controle de quantidade de requisição
            inicio = time()
            sleep(randint(1,5))
            espera = time() - inicio

            print('Request ' + str(count) + ' scrape ' + evento + ' ' + str(espera))

            count += 1

            resultado_individual = get_resultado_individual_detalhado(linha['url'])
            resultado_individual['Categoria'] = linha['categoria']
            resultado_individual['Colocação Geral'] = resultado_individual['Colocação Geral'].split(' ')[0]

            data.append(list(resultado_individual.values()))

            columns = ['Nome', 'N. Peito', 'Equipe', 'Modalidade', 'Sexo', 'Colocação Geral', 'Colocação faixa etária','Tempo bruto', 'Tempo líquido', 'Categoria']

            df = pd.DataFrame(data=data, columns=columns)
            
            df.to_csv(DATASET_FOLDER + "/" + evento + "/resultado/" + nome_arq_ind + ".csv")


In [ ]:
salva_resultado_individual(debug=False, qtd=2000)

meiadorio2019
Procurando resultado novo para baixar...
maratonadorio2019
Procurando resultado novo para baixar...
 baixando1265771
Request 0 scrape maratonadorio2019 1.0010478496551514
 baixando1265796
Request 1 scrape maratonadorio2019 5.001228094100952
 baixando1265798
Request 2 scrape maratonadorio2019 2.001986503601074
 baixando1265800
Request 3 scrape maratonadorio2019 4.00261664390564
 baixando1265818
Request 4 scrape maratonadorio2019 1.0011250972747803
 baixando1265823
Request 5 scrape maratonadorio2019 5.005056858062744
 baixando1265839
Request 6 scrape maratonadorio2019 3.0005011558532715
 baixando1265855
Request 7 scrape maratonadorio2019 2.0021119117736816
 baixando1265860
Request 8 scrape maratonadorio2019 5.003207683563232
 baixando1265861
Request 9 scrape maratonadorio2019 3.0024728775024414
 baixando1265870
Request 10 scrape maratonadorio2019 5.003755331039429
 baixando1265876
Request 11 scrape maratonadorio2019 4.004055023193359
 baixando1265877
Request 12 scrape marat

 baixando1261171
Request 114 scrape maratonadorio2019 5.001264810562134
 baixando1261172
Request 115 scrape maratonadorio2019 3.0022010803222656
 baixando1261187
Request 116 scrape maratonadorio2019 2.0021088123321533
 baixando1261200
Request 117 scrape maratonadorio2019 2.0021133422851562
 baixando1261241
Request 118 scrape maratonadorio2019 1.0011157989501953
 baixando1261244
Request 119 scrape maratonadorio2019 5.004546165466309
 baixando1261279
Request 120 scrape maratonadorio2019 1.0011210441589355
 baixando1261281
Request 121 scrape maratonadorio2019 1.0007715225219727
 baixando1261299
Request 122 scrape maratonadorio2019 4.003015518188477
 baixando1261308
Request 123 scrape maratonadorio2019 2.0020289421081543
 baixando1261333
Request 124 scrape maratonadorio2019 3.0012848377227783
 baixando1261334
Request 125 scrape maratonadorio2019 1.0003974437713623
 baixando1261344
Request 126 scrape maratonadorio2019 5.001073360443115
 baixando1261345
Request 127 scrape maratonadorio2019 3

 baixando1262797
Request 228 scrape maratonadorio2019 5.001706123352051
 baixando1262822
Request 229 scrape maratonadorio2019 5.003401279449463
 baixando1262824
Request 230 scrape maratonadorio2019 2.002112627029419
 baixando1262840
Request 231 scrape maratonadorio2019 4.001333713531494
 baixando1262841
Request 232 scrape maratonadorio2019 1.0010840892791748
 baixando1262867
Request 233 scrape maratonadorio2019 5.0038440227508545
 baixando1262873
Request 234 scrape maratonadorio2019 3.000869035720825
 baixando1262909
Request 235 scrape maratonadorio2019 4.0041210651397705
 baixando1262913
Request 236 scrape maratonadorio2019 3.0014617443084717
 baixando1262916
Request 237 scrape maratonadorio2019 2.0016539096832275
 baixando1262925
Request 238 scrape maratonadorio2019 1.0011026859283447
 baixando1262954
Request 239 scrape maratonadorio2019 4.0041046142578125
 baixando1262956
Request 240 scrape maratonadorio2019 2.0021750926971436
 baixando1262962
Request 241 scrape maratonadorio2019 4.

Request 341 scrape maratonadorio2019 3.0005147457122803
 baixando1264824
Request 342 scrape maratonadorio2019 5.005079984664917
 baixando1264826
Request 343 scrape maratonadorio2019 2.0020911693573
 baixando1264848
Request 344 scrape maratonadorio2019 3.0030763149261475
 baixando1264861
Request 345 scrape maratonadorio2019 2.0020909309387207
 baixando1264867
Request 346 scrape maratonadorio2019 5.000783681869507
 baixando1264869
Request 347 scrape maratonadorio2019 2.0020580291748047
 baixando1264882
Request 348 scrape maratonadorio2019 5.004976272583008
 baixando1264900
Request 349 scrape maratonadorio2019 1.000978946685791
 baixando1264901
Request 350 scrape maratonadorio2019 3.0010488033294678
 baixando1264906
Request 351 scrape maratonadorio2019 1.00105619430542
 baixando1264907
Request 352 scrape maratonadorio2019 1.000645399093628
 baixando1264914
Request 353 scrape maratonadorio2019 5.005082130432129
 baixando1264935
Request 354 scrape maratonadorio2019 2.0011754035949707
 baixa

 baixando1260980
Request 455 scrape maratonadorio2019 4.003957033157349
 baixando1260984
Request 456 scrape maratonadorio2019 2.0003533363342285
 baixando1260986
Request 457 scrape maratonadorio2019 1.0011231899261475
 baixando1260987
Request 458 scrape maratonadorio2019 2.0009686946868896
 baixando1260994
Request 459 scrape maratonadorio2019 3.000640630722046
 baixando1260996
Request 460 scrape maratonadorio2019 5.004408597946167
 baixando1261009
Request 461 scrape maratonadorio2019 4.004053831100464
 baixando1261012
Request 462 scrape maratonadorio2019 5.003578424453735
 baixando1261013
Request 463 scrape maratonadorio2019 5.00506067276001
 baixando1261014
Request 464 scrape maratonadorio2019 2.0020811557769775
 baixando1261022
Request 465 scrape maratonadorio2019 4.00092887878418
 baixando1261028
Request 466 scrape maratonadorio2019 2.0013108253479004
 baixando1261030
Request 467 scrape maratonadorio2019 1.00111985206604
 baixando1261032
Request 468 scrape maratonadorio2019 4.004119

 baixando1261428
Request 569 scrape maratonadorio2019 1.0005931854248047
 baixando1261430
Request 570 scrape maratonadorio2019 4.004136323928833
 baixando1261431
Request 571 scrape maratonadorio2019 1.001129150390625
 baixando1261432
Request 572 scrape maratonadorio2019 4.004126310348511
 baixando1261440
Request 573 scrape maratonadorio2019 3.0010807514190674
 baixando1261458
Request 574 scrape maratonadorio2019 2.0021278858184814
 baixando1261464
Request 575 scrape maratonadorio2019 1.0011355876922607
 baixando1261469
Request 576 scrape maratonadorio2019 3.002192974090576
 baixando1261470
Request 577 scrape maratonadorio2019 2.0021238327026367
 baixando1261473
Request 578 scrape maratonadorio2019 4.004144906997681
 baixando1261475
Request 579 scrape maratonadorio2019 2.0021181106567383
 baixando1261476
Request 580 scrape maratonadorio2019 4.004101514816284
 baixando1261478
Request 581 scrape maratonadorio2019 5.001809120178223
 baixando1261485
Request 582 scrape maratonadorio2019 1.00

 baixando1262006
Request 683 scrape maratonadorio2019 1.001112937927246
 baixando1262007
Request 684 scrape maratonadorio2019 4.004120349884033
 baixando1262012
Request 685 scrape maratonadorio2019 5.005117893218994
 baixando1262017
Request 686 scrape maratonadorio2019 1.0011441707611084
 baixando1262018
Request 687 scrape maratonadorio2019 4.001955032348633
 baixando1262019
Request 688 scrape maratonadorio2019 3.0031344890594482
 baixando1262022
Request 689 scrape maratonadorio2019 1.0005154609680176
 baixando1262024
Request 690 scrape maratonadorio2019 4.004115581512451
 baixando1262026
Request 691 scrape maratonadorio2019 1.0011444091796875
 baixando1262030
Request 692 scrape maratonadorio2019 5.002339839935303
 baixando1262031
Request 693 scrape maratonadorio2019 5.001821517944336
 baixando1262034
Request 694 scrape maratonadorio2019 5.0022571086883545
 baixando1262035
Request 695 scrape maratonadorio2019 3.001307249069214
 baixando1262046
Request 696 scrape maratonadorio2019 1.001

Request 796 scrape maratonadorio2019 4.001495838165283
 baixando1262461
Request 797 scrape maratonadorio2019 5.005116939544678
 baixando1262464
Request 798 scrape maratonadorio2019 5.005119562149048
 baixando1262468
Request 799 scrape maratonadorio2019 3.003119468688965
 baixando1262473
Request 800 scrape maratonadorio2019 4.001934766769409
 baixando1262475
Request 801 scrape maratonadorio2019 4.002719879150391
 baixando1262480
Request 802 scrape maratonadorio2019 3.0031235218048096
 baixando1262482
Request 803 scrape maratonadorio2019 2.0012574195861816
 baixando1262488
Request 804 scrape maratonadorio2019 1.001121997833252
 baixando1262491
Request 805 scrape maratonadorio2019 5.005122661590576
 baixando1262492
Request 806 scrape maratonadorio2019 5.005125284194946
 baixando1262503
Request 807 scrape maratonadorio2019 1.001121997833252
 baixando1262506
Request 808 scrape maratonadorio2019 4.0040366649627686
 baixando1262509
Request 809 scrape maratonadorio2019 4.004120588302612
 baixa